# ENTSO-G Daten

Die Daten der European Network of Transmission System Operators for Gas (ENTSO-G) werden auf dem Transparenzportal veröffentlicht.

Über eine API lassen sich die Daten, sowie Historiendaten abfragen.

Das Projekt unterteilt sich zunächst in 3 Teile:

1. Daten abfragen und speichern
2. Daten aufbereiten
3. Daten visualisieren

Im Gegensatz zu den ENTSO-E Daten gibt es hierbei noch keinerlei Web-Frontends, welche die Daten visualisieren

<h1 id="tocheading">Inhaltsverzeichnis</h1>
<div id="toc"></div>
<script src="http://ajax.googleapis.com/ajax/libs/jquery/1.7.1/jquery.min.js" type="text/javascript"></script>
<script src="https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js"></script>

In [ ]:
%%javascript
//erzeugt Inhaltsverzeichnis
//https://github.com/kmahelona/ipython_notebook_goodies
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

## Begriffserklärungen

* Bidding zones sind die Marktgebiete. Um zwischen Marktgebieten Gas zu transferieren müssen die Gas-Pipelines gebucht werden. 
* Jedes Land hat sein eigenes Marktgebiet (mit Ausnahmen, bspw hat DE bis 2021 noch 2 [NCG und GASPOOL])
* In einem Marktgebiet gibt es verschiedene Operatoren, welche das Gasnetz betreiben
* Es gibt L-Gas und H-Gas, welche unterschiedlich gebucht und transferiert werden
* Jede Pipeline hat einen Entry und einen Exit. Gas fließt immer in eine Richtung

## Bedienung der Plots

Die Plots wurden mit Plotly erstellt.

* Diese Bibliothek erlaubt es durch anklicken in der Legende eine Datenreihe auszublenden.
* Durch ein Doppelklick auf eine Datenreihe kann eine Datenreihe isoliert betrachtet werden. Anschließend können weitere zu betrachtene Datenreihen ausgewählt werden
* Durch ziehen eines Intervalls, kann man einen genaueren Betrachtungsraum auswählen. Ein Doppelklick in das Diagramm springt wieder zum Ursprungsbereich

So gibt es bspw. die Pipeline "Nord Stream"

# Datenbeschaffung

Die Daten können über die API in CSV, JSON oder XML runtergeladen werden.

Tabellarische Daten lassen sich davon am effizientesten als CSV darstellen, weshalb dieses Format gewählt wurde.

Diese werden anschließend in eine Parquet-Datei gespeichert.
Darüber können wir nun über die Historien-Daten zugreifen.

Die Daten der ENTSO-G stehen in diesem Format über die API seit 2017 zur Verfügung.

In [ ]:
import pandas as pd
import plotly.graph_objects as go
aggData = pd.read_csv('https://transparency.entsog.eu/api/v1/AggregatedData.csv?limit=1000')
interconn = pd.read_csv('https://transparency.entsog.eu/api/v1/Interconnections.csv?limit=-1')
# the connectionpoints contain invalid data, as each line ends with an extra seperator
conn = pd.read_csv('https://transparency.entsog.eu/api/v1/connectionpoints.csv?limit=-1',index_col=False)

In [ ]:
import requests
csv = requests.get('https://transparency.entsog.eu/api/v1/connectionpoints.csv?limit=-1')
cleaned = csv.text.replace('commercialType,importFromCountryKey','installation,commercialType,importFromCountryKey')
from io import StringIO
fileLike = StringIO(cleaned)
conn = pd.read_csv(fileLike)

In [ ]:
jsn = requests.get('https://transparency.entsog.eu/api/v1/connectionpoints.json?limit=-1')
conn = pd.DataFrame(jsn.json()['connectionpoints'])


Zum Herunterladen der Daten wurde eine Python Klasse geschrieben, welche die Daten für einen gebenen Bereich von der Api lädt. Diese werden anschließend sowohl in einer SQLite Datenbank, wie auch in Parquet-Dateien mithilfe von Spark gespeichert.

Die Datenstruktur sieht wie folgt aus:

![Datenbank-Struktur](Diagrams-ENTSO-G.png)

Um auf die Daten nachfolgend zuzugreifen verwenden wir Spark

In [ ]:
import findspark

findspark.init()
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pandas as pd

conf = SparkConf().setAppName('entsog').setMaster('local')
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

In [ ]:
sparkfolder='data/entsog_spark'

Nun Lesen wir die Daten aus dem Spark-Daten-Ordner

In [ ]:
interconnections = spark.read.parquet(f"{sparkfolder}/Interconnections.parquet")
balancingzones = spark.read.parquet(f"{sparkfolder}/balancingzones.parquet")
opdata = spark.read.parquet(f"{sparkfolder}/operationaldata")
oppointdirs = spark.read.parquet(f"{sparkfolder}/operatorpointdirections.parquet")
operas = spark.read.parquet(f"{sparkfolder}/operators.parquet")
connectionpoints = spark.read.parquet(f"{sparkfolder}/connectionpoints.parquet")

In [ ]:
# generalRemarks must be deleted as column contains different datatypes

# structure files in parquet, so that hierarchy is better:
#%time opdata.drop('generalRemarks').write.partitionBy(['year','month']).parquet(f"{sparkfolder}/operationaldata_cleaned")

In [ ]:
# converting time column to be grouped by a given period works as easy as this:
from pyspark.sql.functions import date_trunc
opdata = ( opdata.withColumn("day", date_trunc('day',"periodFrom"))
          .withColumn("hour", date_trunc('hour',"periodFrom"))
         )

In [ ]:
# selecting only necessary stuff from static tables
inter= interconnections.select(['pointlabel','pointTpMapX','pointTpMapY','fromCountryKey','fromoperatorkey','frombzlabel','frombzlabel','tooperatorlabel','toCountryKey','pointkey','toPointKey','topointlabel']).toPandas()
balancing= balancingzones.select(['bzLabel','tpMapX','tpMapY']).toPandas()
conne = (connectionpoints.withColumn("long", connectionpoints.tpMapX)
        .withColumn("lat", connectionpoints.tpMapY)
        .select(['pointkey','lat','long','infrastructureLabel']).toPandas()
       )
ops=operas.select(['operatorkey','operatorLabel','operatorCountryKey','operatorTypeLabelLong']).toPandas()

In [ ]:
# show small table of available operators
import plotly.graph_objects as go
fig = go.Figure(data=[go.Table(
    header=dict(values=['operatorkey','Betreiber Bezeichnung','Land des Betreibers','Art des Betreibers'],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[ops.operatorKey, ops.operatorLabel, ops.operatorCountryKey, ops.operatorTypeLabelLong],
               fill_color='lavender',
               align='left'))
])

fig.show()

Zu sehen ist hier eine Liste aller Operators und ihrer Länder.  
Während diese Liste nicht durchsuchbar ist, ist die DataTable im Dashboard auch durchsuchbar.

Die Interconnections und Bidding Zonen haben sogar auch X und Y Daten. Diese sind leider nicht in Koordinaten-Form zu bringen. Keine Projektion oder Verdrehung brachte ein ordentliches Bild.

Das Problem wurde anschließend gelöst, indem die Map-Tile Bilder direkt von dem ENTSO-G Server herunter geladen wurden und das Kartenmaterial nun selbst lokal gehostet wird. Das Erstellen der Grafiken ist nachfolgend beschrieben:

## Kartenmaterialbeschaffung und Visualisierungen

Die Kartendaten laden wir direkt von deren Server runter, nachdem festgestellt wurde, dass die X und Y Punkte nicht auf eine klassische Merkator Projektion passen. <br/>
Hier gibt es 5 Zoomstufen, für die jeweils Kartenmaterial als Bilddatei heruntergeladen werden kann.<br/>
Außerdem gibt es verschiedene Overlays für Karten, welche angezeigt werden können.

Leider sind die Daten im TMS-Format, das heißt, dass die Y-Achse invertiert ist.

Für mehr Informationen, siehe hier: https://gist.github.com/tmcw/4954720

Dort ist auch beschrieben, wie man diese Daten in das XYZ konvertieren kann. Das ist notwendig, da Plotly TMS leider [nicht unterstützt](https://github.com/plotly/plotly.py/issues/1610)

In [ ]:
layers = []

routes = ['countries_zones','pipelines_small_medium_large','pipelines_medium_large','pipelines_large','drilling_platforms','gasfields','projects','country_names']
routes = ['countries_zones','pipelines_small_medium_large','country_names']

for route in routes:
    layers.append({   "below": 'traces',
                "sourcetype": "raster",
                "sourceattribution": '<a href="https://transparency.entsog.eu/#/map">ENTSO-G Data</a>',
                "source": [
                    "https://datensch.eu/cdn/entsog/"+route+"/{z}/{x}/{y}.png"]})

### Karte der Grenz-Verbindungen

In [ ]:
inter_border= inter[inter['fromCountryKey'] !=inter['toCountryKey']]

import plotly.express as px
fig = px.scatter_mapbox(inter_border, lat="pointTpMapY", lon="pointTpMapX", hover_name="pointlabel", color='fromCountryKey',hover_data=["pointKey", "fromCountryKey",'fromoperatorkey','tooperatorlabel',"toCountryKey"],
                        zoom=1, height=600)
fig.update_layout(title="Karte der Verbindungspunkte mit Grenzübergang",mapbox_style="white-bg",mapbox_layers=layers,margin={"r":0,"l":0,"b":0})
fig.show()

### Karte aller Verbindungspunkte

Über die HoverData können die sichtbaren Felder ausgewählt werden.

Im finalen Dashboard wird stattdessen der Anzeigetext verändert

In [ ]:
import plotly.express as px
#import plotly.graph_objects as go
fig = px.scatter_mapbox(conne, lat="lat", lon="long", hover_name="pointKey", hover_data=["infrastructureLabel"],color="infrastructureLabel",
                        zoom=1, height=600,title='Karte der Verbindungspunkte')
fig.update_layout(mapbox_style="white-bg",mapbox_layers=layers,margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Karte der Marktgebiete

In [ ]:
import plotly.express as px
#import plotly.graph_objects as go
fig = px.scatter_mapbox(balancing, lat="tpMapY", lon="tpMapX", hover_name="bzLabel", #color='bzLabel',
                        color_discrete_sequence=["fuchsia"], zoom=1, height=600,title='Karte der Marktgebiete')
fig.update_layout(title="Karte der Marktgebiete", mapbox_style="white-bg",mapbox_layers=layers,margin={"r":0,"l":0,"b":0})
fig.show()


Man könnte hier nun auch ein GeoJSON erstellen um eine Chloropleth Karte der Gebiete zu erstellen, hierzu müsste man auf https://geojson.io das Kartenmaterial als Layer einbinden und die Polygone der Marktgebiete einzeichnen.

# Auswertungen

Neben diesem Nebenschauplatz der Kartografie, besteht natürlich auch der Hauptteil, welcher das Herunterladen, strukturieren und abfragen der Daten beinhaltet.

Die Daten können nun über die API herunter geladen werden und in einer Datenbank oder Parquet-Datei abgespeichert werden.

Insgesamt haben die Daten bereits eine gute Qualität und ein relationales Format, dennoch gibt es einige Fallstricke die man beachten muss.
So gibt es Pipe-in-Pipe Verbindungen, welche bei Nichtbeachtung eine doppelte Buchung abgeben.

Nach herausfiltern gibt es an diesen Punkten dennoch größere Unstimmigkeiten die bisher nicht erklärt werden konnten. Dazu wird eine tiefergehende Datenanalyse benötigt.

Diese ist jedoch nicht Teil des Projekts, kann mit dem bereitgestellten Dashboard jedoch gut durchgeführt werden.

Kommen wir erstmal zurück zu den Gasdatenflüssen, die wir Visualisieren möchten.

Die Daten stehen uns erstmal zur Verfügung im folgenden Intervall:

In [ ]:
beg=opdata.select('periodFrom').rdd.min()
end=opdata.select('periodFrom').rdd.max()

print('Datensequenz von {} bis {}'.format(beg[0],end[0]))

In [ ]:
# erzeugen der operator daten
%time operators=opdata.select('operatorkey','operatorLabel').distinct().toPandas()

In [ ]:
from pyspark.sql.functions import date_trunc
# pandas data frame (pdf)
pdf =(
    opdata.withColumn("day", date_trunc('day',"periodFrom"))
    .withColumn("month", date_trunc('month',"periodFrom"))
    .withColumn("year", date_trunc('year',"periodFrom"))
).select('operatorLabel','operatorkey','year','month','value').groupby(['operatorkey','operatorLabel','month']).sum('value')

# wird erst im toPandas ausgeführt. Dauert jedoch seehr lange
%time pdf = pdf.toPandas()

In [ ]:
operators.sort_values('operatorKey')
singleTso = (opdata.withColumn("day", date_trunc('day',"periodFrom"))
     .select(['value','day'])
     .filter('operatorKey = "DE-TSO-0013"')
     .groupby('day').sum('value').sort("day")
    )
%time singleTso=  singleTso.toPandas()

In [ ]:
import plotly.express as px
print(operators[operators['operatorKey']=='DE-TSO-0013'])

In [ ]:
px.line(singleTso,x='day',y='sum(value)',title='Gefördertes Gas von Jordgas Transport',labels={'day':'Zeit','sum(value)':'gefördertes Gas in MWh'})

In [ ]:
monthly_data= pdf.pivot(index='month', columns='operatorLabel',values='sum(value)')
px.line(monthly_data, title='Gefördertes Gas pro Monat in kWh nach Betreiber',labels={'month':'Zeit','value':'gefördertes Gas in kWh'})

Hier ist zu sehen, dass es in den Daten auch größere Unstimmigkeiten gibt, die durch fehlerhafte Dateneingaben in die Transparenz-Plattform entstanden sind

## Verlauf der Nordstream

Auslastung

Die Nord Stream hat als interne Bezeichnung den pointKey ITP-00120.
Sie fördert Gas von RU nach DE.
Genau genommen zu den PointKeys 

* ITP-00491 (Greifswald)
* ITP-00250 (Greifswald)
* ITP-00251 (Gw-Opal)? Wird hier aus der OPAL eingespeist? Unsinn?
* ITP-00247 (Greifswald)
* ITP-00297 (Greifswald)
* ITP-00454 (Lubmin, reguliert)

Diese Punkte erhalten Gas von der Nordstream und liegen allesamt in Deutschland.

Die BZ erhält man aus den Interconnections, allerdings nur für die (tote) Leitung von RU nach DE: GASPOOL
Die Datenlage mit Punkten außerhalb der EU ist hier sehr schlechter


### Relevante Knoten

In [ ]:
inter[(inter.tooperatorlabel=='Nord Stream')|(inter.fromoperatorkey=='Nord Stream')]

In [ ]:
pips =list(inter[(inter.tooperatorlabel=='Nord Stream')|(inter.fromoperatorkey=='Nord Stream')].toPointKey)
pips

In [ ]:
%time d= opdata.filter(opdata.pointKey.isin(pips)).groupby(['pointkey','directionkey','day']).sum('value').toPandas()

In [ ]:
%time january = opdata.filter('"2018-01-01" <= day and day < "2019-02-01"').filter(opdata.pointKey.isin(pips)).groupby(['pointkey','pointlabel','directionkey','hour']).sum('value').toPandas()

In [ ]:
p = january.pivot(index='hour', columns=['pointlabel','directionkey'],values='sum(value)')
p.columns=list(map(lambda x: '_'.join(x),p.columns))
fig = px.line(p,title='Durchsatz der Nordstream Pipeline')
fig.update_layout( xaxis_title='Zeit',
                   yaxis_title='Durchsatz in kWh pro Tag')
fig.show()

## Visualisierung des physischen Flusses als Widget mit SQLite

Nun möchten wir ein etwas interaktiveres Widget erstellen, bei dem man die Zone auswählen kann für die man den physischen Fluss visualisieren möchte.

Der nachfolgende Source-Code beschreibt das Filtern und auswählen der Daten mithilfe von SQLite.

Die Daten pro Zone werden berechnet, indem die Summe aller Pipelines in dieser Zone berechnet wird. Das ist sehr aufwändig

In [ ]:
import sqlite3
from contextlib import closing
import pandas as pd
from entsog_data_manager import Filter
from datetime import datetime, date, timedelta

ftime = {'day': '%Y-%m-%d',
         'month': '%Y-%m-01',
         'year': '%Y-01-01',
         'hour': '%Y-%m-%d %H:00:00',
         'minute': '%Y-%m-%d %H:%M:00'}

def timeFilter(filt: Filter):
    return f'"{filt.begin.strftime("%Y-%m-%d")}" < periodFrom and periodFrom < "{filt.end.strftime("%Y-%m-%d")}"'

In [ ]:
def getData(name,filt: Filter, direction=None):
    conn = sqlite3.connect('data/Firm Technical.db')
    
    selectString = f"strftime('{ftime[filt.groupby]}', periodFrom) as time, sum(value) as 'value'"
    whereString = f" directionkey='{direction}' and adjacentSystemsLabel='{name}' and {timeFilter(filt)}"
    groupString = f'strftime("{ftime[filt.groupby]}", "time")'
    
    if direction:
        query = f"SELECT {selectString} FROM AggregatedData WHERE {whereString} group by  {groupString}"
    else:
        query = "select value-exit_value as diff,a.periodFrom,value, exit_value from (SELECT periodFrom,value FROM AggregatedData WHERE directionkey='entry' and adjacentSystemsLabel='"+name+"') a"
        query += " join (select periodFrom,value as exit_value from AggregatedData where directionkey='exit' and adjacentSystemsLabel='"+name+"') b on a.periodFrom = b.periodFrom"
        #query += " where a.periodFrom < '2018-01-01'"
        #and operatorKey='BE-TSO-0001'
    df = pd.read_sql_query(query,conn)
    #df['begin']=pd.to_datetime(df['periodFrom']).dt.to_period('M').dt.to_timestamp()
    #df['begin']=pd.to_datetime(df['periodFrom']).dt.floor('d')



    return df

def update(name,filt):
    entrys = getData(name,filt,'entry')
    exits = getData(name,filt,'exit')
    diff = entrys['value']-exits['value']
    entrys['diff']= diff
    entrys['exit']=exits['value']
    return entrys

#operatorKey
with closing(sqlite3.connect('data/Firm Technical.db')) as conn:
    #operators2 = pd.read_sql_query('select distinct operatorKey,operatorLabel from AggregatedData',conn)
    #operators = pd.read_sql_query('select distinct adjacentSystemsKey,adjacentSystemsLabel from AggregatedData',conn)
    
    #operators.to_sql('operators',conn)
    #operators2.to_sql('operators2',conn)
    operators = pd.read_sql('select * from operators', conn)
    operators2 = pd.read_sql('select * from operators2',conn)
    
filt = Filter(datetime(2018, 4, 1), datetime(2018, 7, 2), 'day')
%time data = update('NCG',filt)

In [ ]:
import plotly.graph_objects as go
from ipywidgets import widgets

month = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Month:',
    continuous_update=False
)

use_date = widgets.Checkbox(
    description='Date: ',
    value=True,
)

container = widgets.HBox(children=[use_date, month])

textbox = widgets.Dropdown(
    description='Operators:   ',
    value=operators['operators'][0],
    options=operators['operators'].tolist()
)


trace1 = go.Scatter(x=data['time'],y=data['value'],name='importiertes Gas')#,mode='markers')
trace2 = go.Scatter(x=data['time'],y=data['exit'],name='exportiertes Gas')#,mode='markers')
trace3 = go.Scatter(x=data['time'],y=data['diff'],name='dem Gebiet beigefügtes Gas')#,mode='markers')
               #hover_name="bzLabel", hover_data=["bzTooltip", "id"])    
# Assign an empty figure widget with two traces
#trace1 = go.Histogram(x=df['arr_delay'], opacity=0.75, name='Arrival Delays')
#trace2 = go.Histogram(x=df['dep_delay'], opacity=0.75, name='Departure Delays')
g = go.FigureWidget(data=[trace1, trace2,trace3],
                    layout=go.Layout(
                        title=dict(
                            text='ENTSOG Physical Flow'
                        ),
                        xaxis=dict(title='Datum'),
                        yaxis=dict(title='gefördertes Gas in kWh')
                        #barmode='overlay'
                    ))

def validate():
    if textbox.value in operators['operators'].to_list():
        return True
    else:
        return False

import time
def response(change):
    if validate():
        print('updating...')
        t = time.time()
        temp_df = update(textbox.value,filt=filt)
        x1 = temp_df['value']
        x2 = temp_df['exit']
        x3 = temp_df['diff']
        print('finished updating', time.time()-t)
        with g.batch_update():
            g.data[0].y = x1
            g.data[1].y = x2
            g.data[2].y = x3
            #g.layout.xaxis.title = 'Datum'
            #g.layout.yaxis.title = 'gefördertes Gas in kWh'
    else:
        print('Invalid name: ', textbox.value)

textbox.observe(response, names="value")

In [ ]:
widgets.VBox([widgets.HBox([textbox]),
              g])

Das Widget ist in der HTML Version leider nicht verfügbar

# Analysen

Nachfolgend wurden einige Analysen mithilfe des SQLite Zugriffsobjekts erstellt.

Die Performance ist hierbei erheblich besser als mit Spark/Parquet

## Visualisierungen des Physical Flows

In der Datenbank enthält jeder Eintrag einen zugehörigen Wert und Operator. So ist es möglich, dass nicht alle operators zu jeder Stunde einen Eintrag haben.

Die Darstellung mit einer Spalte pro Operator wird durch pandas.pivot erzeugt.
Damit kann sehr einfach ein Plot erzeugt werden.

Nachfolgend wird der physische Fluss von mehreren Operators visualisiert, sowie am Punkt Eynatten in Lichtenbusch

In [ ]:
from entsog_sqlite_manager import EntsogSQLite
from entsog_data_manager import Filter
from datetime import datetime, date, timedelta
import pandas as pd
import plotly.express as px

In [ ]:
entsog = EntsogSQLite('data/entsog.db')
operators = entsog.operators()

start = datetime(2018, 7, 1)
end = datetime(2018, 7, 22)
group = 'hour'
filt = Filter(start, end, group)
balzones = entsog.balancingzones()
intercon = entsog.interconnections()
cpp = entsog.connectionpoints()
#gen = generation.melt(var_name='kind', value_name='value',ignore_index=False)

# GRTD, GUD, GTG, Fluxys
operatorKeys = ['DE-TSO-0004', 'DE-TSO-0007', 'DE-TSO-0005', 'DE-TSO-0006']

phy = entsog.operationaldata(operatorKeys, filt)
piv = phy.pivot(columns=['operatorkey', 'directionkey'], values='value')
piv.plot(rot=45, title='Physical flow of selected operators by direction',figsize=(10,6))

### Flow der Punkte in Eynatten/Lichtenbusch

In [ ]:
point = entsog.operationaldataByPoints(
        ['ITP-00043', 'ITP-00111'], Filter(start, end, group), ['pointkey', 'directionkey'])
point['point'] = point['pointlabel']+' ' + \
    point['directionkey']+' '+point['indicator']
point['value'] = point['value']/1e6
piv2 = point.pivot(columns=['point'], values='value')
piv2.plot(rot=45, title='Physical flow of selected points by direction',figsize=(15,6))

### Physischer Fluss Portugal, nach Art des Messpunktes

Die Abkürzungen sind hier:
* ITP - Inter Transmission Point
* LNG - LNG Terminal (Flüssigerdgas)
* UGS - Underground storage
* DIS - Distribution
* FNC - Final Consumers

In [ ]:
end = datetime(2018, 7, 2)
filt = Filter(start, end, 'hour')
operatorKeys = entsog.operatorsByBZ('Italy')

operatorKeys = entsog.operatorsByBZ('Portugal')
bil = entsog.bilanz(operatorKeys, filt)
bil.plot(rot=45, title='netto Physical flow of Portugal seperated by distribution kind',figsize=(10,6))

### Physischer Fluss zu Nachbar-Regionen von GASPOOL

In [ ]:
operatorKeys = entsog.operatorsByBZ('GASPOOL')
c = entsog.crossborder(operatorKeys, filt)
c.plot(rot=45, title='Transferred physical flow of GASPOOL',figsize=(10,6))

## Abweichungen des Physical Flows zu den Allokationen am Beispiel NCG

Neben dem Physischen Fluss (dem was tatsächlich durchgegangen ist), gibt es auch noch die Allokationen (was angemeldet wurde, was durchläuft). 
In der Theorie muss hier bei Summenbildung ähnliche Werte zustande kommen.

In der Praxis kommt es hierbei jedoch zu Unstimmigkeiten und Verschiebungen. Jedoch ist eine generelle Ähnlichkeit des Verlaufs ersichtlich.

In [ ]:
start = datetime(2018, 7, 1)
end = datetime(2018, 7, 22)
group = 'hour'
filt = Filter(start, end, group)
bz = 'NCG'
operatorKeys = entsog.operatorsByBZ(bz)
p = entsog.operationaldata(operatorKeys, filt)
a = entsog.operationaldata(operatorKeys, filt, table='Allocation')


a = a.pivot(columns=['directionkey'], values='value')
p = p.pivot(columns=['directionkey'], values='value')
if 'entry' in a.columns and 'exit' in a.columns:
    a['usage'] = a['entry']-a['exit']

if 'entry' in p.columns and 'exit' in p.columns:
    p['usage'] = p['entry']-p['exit']

a.columns = list(map(lambda x: 'allocation '+str(x), a.columns))
p.columns = list(map(lambda x: 'physical '+str(x), p.columns))
ap = pd.concat([a, p], axis=1)
px.line(ap, title='Physical Flow for '+bz)

## Abweichungen nach Messpunktart

Visualisiert man sich die Allokationen nach Messpunkt art, sieht man, dass die Unstimmigkeiten zwischen Allokationen und physischem Fluss hauptsächlich bei den Verbindungspunkten zu anderen Zonen (ITP) entstehen.

Jedoch gibt es auch bei den Untergrundspeichern unstimmigkeiten

In [ ]:
p = entsog.bilanz(operatorKeys, filt)
a = entsog.bilanz(operatorKeys, filt, table='Allocation')

a.columns = list(map(lambda x: x+' alloc', a.columns))
p.columns = list(map(lambda x: x+' phy', p.columns))
g = pd.concat([a, p], axis=1)
px.line(g, title='Physical Flow by production kind '+bz)

## Abweichungen nach Land des Transfers

In einer Visualisierung des Physischen Flusses und den Allokationen nach Transferland, sieht man dass 
Vergleicht man den Physischen Fluss mit den Allokationen, sieht man, dass starke Abweichungen bei den Transferdaten von Norwegen entstehen.

Außerdem gibt es zu FNC bspw keine Allokationsdaten, sodass auch hierbei Unstimmigkeiten auftreten

In [ ]:
p = entsog.crossborder(operatorKeys, filt)
a = entsog.crossborder(operatorKeys, filt, table='Allocation')
a.columns = list(map(lambda x: x+' alloc', a.columns))
p.columns = list(map(lambda x: x+' phy', p.columns))

g = pd.concat([p, a], axis=1)
px.line(g, title='Transmission flows '+bz)

Durch doppelklicken auf einen Eintrag in der Legende und anschließendem anklicken der zugehörigen Allokationen, kann man zu einzelnen Linien phy und alloc vergleichen

## Import + Prod – Export == Verbrauch + Entry UGS – Exit UGS

Nun soll die Rechnung Import + Prod – Export == Verbrauch + Entry UGS – Exit UGS überprüft werden.

Das ist logisch betrachtet stimmig, da alles was das Land nicht verlässt (linke Seite) entweder gespeichert oder verbraucht wird

In [ ]:
# helper
def fixMissing(x):
    if not 'PRD' in list(x.columns):
        x['PRD']=0
    if not 'UGS' in list(x.columns):
        x['UGS']=0
    if not 'ITP' in list(x.columns):
        x['ITP']=0   
        
bz = 'NCG' #'TRF'
operatorKeys = entsog.operatorsByBZ(bz)
start = datetime(2018, 4, 1)
end = datetime(2018, 7, 22)
group = 'day'
filt = Filter(start, end, group)


bil_p = entsog.bilanz(operatorKeys, filt)
bil_a = entsog.bilanz(operatorKeys, filt, table='Allocation')
p = entsog.operationaldata(operatorKeys, filt)
a = entsog.operationaldata(operatorKeys, filt, table='Allocation')

a = a.pivot(columns=['directionkey'], values='value')
p = p.pivot(columns=['directionkey'], values='value')
if 'entry' in a.columns and 'exit' in a.columns:
    a['usage'] = a['entry']-a['exit']

if 'entry' in p.columns and 'exit' in p.columns:
    p['usage'] = p['entry']-p['exit']
#ap = pd.concat([a, p], axis=1)        
fixMissing(bil_a)
fixMissing(bil_p)

In [ ]:
ap = pd.DataFrame()
if not a.empty:
    ap['left alloc'] = bil_a['PRD']+bil_a['ITP']
    ap['right alloc'] = a['usage']-bil_a['UGS']
    ap['diff alloc'] = ap['left alloc']-ap['right alloc']

if not p.empty:
    ap['left phys'] = bil_p['PRD']+bil_p['ITP']
    ap['right phys'] = p['usage']-bil_p['UGS']
    ap['diff phys'] = ap['left phys']-ap['right phys']
#ap = pd.concat([ap['left alloc'], right_alloc], axis=1)
#ap = pd.concat([ap['left alloc'], right_alloc,left_phys, right_phys], axis=1)

ap['diff diff'] = ap['diff phys']-ap['diff alloc']

px.line(ap, title='Physical Flow for '+bz)

Hier sieht man, dass die Rechnung für NCG im ausgewählten Intervall aufgeht (bis auf kleinere Abweichungen).

## Erzeugnisse nach Land

Eine weitere Fragestellung war der Vergleich von Allokationen und Fluss  nach Erzeugnisse pro Zone, Das entspricht der obigen linken Seite.

In [ ]:
start = datetime(2018, 4, 1)
end = datetime(2018, 4, 22)
group = 'hour'
filt = Filter(start, end, group)

In [ ]:
countries = ['NCG','GASPOOL','TRF','Austria']
ap = pd.DataFrame()
for c in countries:
    operatorKeys = entsog.operatorsByBZ(c)
    bil_p = entsog.bilanz(operatorKeys, filt)
    bil_a = entsog.bilanz(operatorKeys, filt, table='Allocation')

    fixMissing(bil_a)
    fixMissing(bil_p)
    
    ap[c+' alloc'] = bil_a['PRD']+bil_a['ITP']
    ap[c+' phys'] = bil_p['PRD']+bil_p['ITP']        
    

In [ ]:
px.line(ap, title='Physical Flow for '+bz)

# Ausblick

## Flussdiagramm

nach Datum über die Zeit oder als Übergangsmatrix/Sankey-Diagramm

Gruppierung nach Land/Marktgebiet


Visualisieren wo Gas benötigt wird und wo es nur geschickt wird

Visualisieren der Gas-Pipeline (Kartenfehler finden)

## Anteil Gasversorgung eines Operators am Marktgebiet

Erzeugen eines DashBoards der vorhandenen Daten, ähnlich wie Energy-charts.info

In einem Marktgebiet kaufen und verkaufen verschiedene Operatoren Gas.
Pro Marktgebiet kann man jeweils für Käufe und Verkäufe die Menge nach Operator aufschlüsseln

Alles weitere findet sich dann im großen Dashboard:

https://entso.nowum.fh-aachen.de/entsog